To DO:


1) Update ground counts when avail (DONE!)
2) Update the average values IXI file when avail (DONE!)




In [1]:
import pandas as pd
import numpy as np
import sys 
sys.path.append('/user-home/1014/code')
from eval_functions import *
#from pdp import *
import lightgbm as lgb

from joblib import Parallel, delayed


pd.set_option("display.max_columns",501)

pd.set_option("display.max_rows",501)

%matplotlib inline

import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import ExtraTreesRegressor
import xgboost as xgb
import xgbfir
from pyearth import Earth
from sklearn.metrics import roc_auc_score, r2_score
from sklearn.model_selection import cross_val_predict, GridSearchCV, StratifiedKFold, train_test_split, KFold 

from sklearn.externals import joblib

#!pip install --user /user-home/py_packages/mlxtend-0.9.0.tar.gz

from sklearn.preprocessing import StandardScaler


from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

import pyspark as spark
from pyspark.sql import SparkSession, functions as F

from pyspark.sql.types import (
    StringType,
    LongType,
    DoubleType,
    StructField,
    StructType,FloatType
)



<h3>MDM Spark Processing</h3>

In [2]:

mdm = (sqlContext.read.format("com.databricks.spark.csv").options(header= "true",delimiter=',', inferSchema="true").load('/user-home/1014/checking_models/score/july18/MDM.csv'))
mdm=(mdm.select('DATE_KEY','census_tract','CONS_BAL_AMT','DEPOSIT_BAL_AMT','SAVINGS_BAL_AMT','CHECKING_BAL_AMT','TOTAL_CROSS_SELL','INVESTMENT_BAL_AMT', 'block_group','CHECKING_IND', 'LESS_24_MONTHS_TENURE','LESS_12_MONTHS_TENURE','A01','A02','A03','A04','B05','B06','B07','B08','C09','C10','C11','C12','D13','D14','D15','D16','E17','E18','E19','E20','F21','F22','F23','F24','G25','G26','G27','G28','H29','H30','H31','I32','J33','J34','J35','J36','K37','K38','K39','L40','L41','L42','M43','M44','M45','M46','M47','N48','N49','N50','N51','O52','O53','P54','P55','P56','P57','Q58','Q59','Q60','R61'))



In [3]:
def median(values_list):
    med = np.median(values_list)
    return float(med)
udf_median = F.udf(median, FloatType())

#group_df = df.groupby(['a', 'd'])
#df_grouped = group_df.agg(udf_median(func.collect_list(col('c'))).alias('median'))

agged_CT=mdm.groupby('DATE_KEY','census_tract').agg(F.mean('CONS_BAL_AMT').alias('CT_MEAN_CONS_BAL_AMT'),
F.mean('DEPOSIT_BAL_AMT').alias('CT_MEAN_DEPOSIT_BAL_AMT'),
F.mean('SAVINGS_BAL_AMT').alias('CT_MEAN_SAVINGS_BAL_AMT'),
F.mean('CHECKING_BAL_AMT').alias('CT_MEAN_CHECKING_BAL_AMT'),
F.mean('TOTAL_CROSS_SELL').alias('CT_MEAN_TOTAL_CROSS_SELL'),
F.mean('INVESTMENT_BAL_AMT').alias('CT_MEAN_INVESTMENT_BAL_AMT'),
udf_median(F.collect_list(F.col('CONS_BAL_AMT'))).alias('CT_MED_CONS_BAL_AMT'),
udf_median(F.collect_list(F.col('DEPOSIT_BAL_AMT'))).alias('CT_MED_DEPOSIT_BAL_AMT'),
udf_median(F.collect_list(F.col('SAVINGS_BAL_AMT'))).alias('CT_MED_SAVINGS_BAL_AMT'),
udf_median(F.collect_list(F.col('CHECKING_BAL_AMT'))).alias('CT_MED_CHECKING_BAL_AMT'),
udf_median(F.collect_list(F.col('TOTAL_CROSS_SELL'))).alias('CT_MED_TOTAL_CROSS_SELL'),
udf_median(F.collect_list(F.col('INVESTMENT_BAL_AMT'))).alias('CT_MED_INVESTMENT_BAL_AMT'),
                                                   


F.sum(F.col('I32')+
F.col('J33')+
F.col('J34')+
F.col('J35')+
F.col('J36')+
F.col('K37')+
F.col('K38')+
F.col('K39')+
F.col('L40')+
F.col('L41')+
F.col('L42')+
F.col('O52')+
F.col('O53')+
F.col('R61')).alias('mdm_fe_ct'),
                                                    
F.sum(F.col('E17')+
F.col('E18')+
F.col('E19')+
F.col('E20')+
F.col('F21')+
F.col('F22')+
F.col('F23')+
F.col('F24')+
F.col('G25')+
F.col('G26')+
F.col('G27')+
F.col('G28')+
F.col('H29')+
F.col('H30')+
F.col('H31')+
F.col('N48')+
F.col('N49')+
F.col('N50')+
F.col('N51')+
F.col('Q58')+
F.col('Q59')+
F.col('Q60')).alias('mdm_ma_ct'),
                                                    
F.sum(F.col('A03')+
F.col('A04')+
F.col('B06')+
F.col('B08')+
F.col('C11')+
F.col('C12')+
F.col('D14')+
F.col('D16')+
F.col('M45')+
F.col('M46')+
F.col('M47')+
F.col('P56')).alias('mdm_mm_ct'),
                                                   
F.sum(F.col('A01')+
F.col('A02')+
F.col('B05')+
F.col('B07')+
F.col('C09')+
F.col('C10')+
F.col('D13')+
F.col('D15')+
F.col('M43')+
F.col('M44')+
F.col('P54')+
F.col('P55')+
F.col('P57')).alias('mdm_ub_ct')
)                                                 
                                  

agged_BG=mdm.groupby('DATE_KEY','block_group').agg(F.mean('CONS_BAL_AMT').alias('BG_MEAN_CONS_BAL_AMT'),
F.mean('DEPOSIT_BAL_AMT').alias('BG_MEAN_DEPOSIT_BAL_AMT'),
F.mean('SAVINGS_BAL_AMT').alias('BG_MEAN_SAVINGS_BAL_AMT'),
F.mean('CHECKING_BAL_AMT').alias('BG_MEAN_CHECKING_BAL_AMT'),
F.mean('TOTAL_CROSS_SELL').alias('BG_MEAN_TOTAL_CROSS_SELL'),
F.mean('INVESTMENT_BAL_AMT').alias('BG_MEAN_INVESTMENT_BAL_AMT'),
udf_median(F.collect_list(F.col('CONS_BAL_AMT'))).alias('BG_MED_CONS_BAL_AMT'),
udf_median(F.collect_list(F.col('DEPOSIT_BAL_AMT'))).alias('BG_MED_DEPOSIT_BAL_AMT'),
udf_median(F.collect_list(F.col('SAVINGS_BAL_AMT'))).alias('BG_MED_SAVINGS_BAL_AMT'),
udf_median(F.collect_list(F.col('CHECKING_BAL_AMT'))).alias('BG_MED_CHECKING_BAL_AMT'),
udf_median(F.collect_list(F.col('TOTAL_CROSS_SELL'))).alias('BG_MED_TOTAL_CROSS_SELL'),
udf_median(F.collect_list(F.col('INVESTMENT_BAL_AMT'))).alias('BG_MED_INVESTMENT_BAL_AMT'),


F.sum(F.col('I32')+
F.col('J33')+
F.col('J34')+
F.col('J35')+
F.col('J36')+
F.col('K37')+
F.col('K38')+
F.col('K39')+
F.col('L40')+
F.col('L41')+
F.col('L42')+
F.col('O52')+
F.col('O53')+
F.col('R61')).alias('mdm_fe_bg'),
                                                    
F.sum(F.col('E17')+
F.col('E18')+
F.col('E19')+
F.col('E20')+
F.col('F21')+
F.col('F22')+
F.col('F23')+
F.col('F24')+
F.col('G25')+
F.col('G26')+
F.col('G27')+
F.col('G28')+
F.col('H29')+
F.col('H30')+
F.col('H31')+
F.col('N48')+
F.col('N49')+
F.col('N50')+
F.col('N51')+
F.col('Q58')+
F.col('Q59')+
F.col('Q60')).alias('mdm_ma_bg'),
                                                    
F.sum(F.col('A03')+
F.col('A04')+
F.col('B06')+
F.col('B08')+
F.col('C11')+
F.col('C12')+
F.col('D14')+
F.col('D16')+
F.col('M45')+
F.col('M46')+
F.col('M47')+
F.col('P56')).alias('mdm_mm_bg'),
                                                   
F.sum(F.col('A01')+
F.col('A02')+
F.col('B05')+
F.col('B07')+
F.col('C09')+
F.col('C10')+
F.col('D13')+
F.col('D15')+
F.col('M43')+
F.col('M44')+
F.col('P54')+
F.col('P55')+
F.col('P57')).alias('mdm_ub_bg')
)                                                 
     
                                                

agged_CT_CHK =mdm.filter(mdm.CHECKING_IND==1).groupby('DATE_KEY','census_tract').agg(F.mean('CONS_BAL_AMT').alias('CT_CHK_MEAN_CONS_BAL_AMT'),
F.mean('DEPOSIT_BAL_AMT').alias('CT_CHK_MEAN_DEPOSIT_BAL_AMT'),
F.mean('SAVINGS_BAL_AMT').alias('CT_CHK_MEAN_SAVINGS_BAL_AMT'),
F.mean('CHECKING_BAL_AMT').alias('CT_CHK_MEAN_CHECKING_BAL_AMT'),
F.mean('TOTAL_CROSS_SELL').alias('CT_CHK_MEAN_TOTAL_CROSS_SELL'),
F.mean('INVESTMENT_BAL_AMT').alias('CT_CHK_MEAN_INVESTMENT_BAL_AMT'),
udf_median(F.collect_list(F.col('CONS_BAL_AMT'))).alias('CT_CHK_MED_CONS_BAL_AMT'),
udf_median(F.collect_list(F.col('DEPOSIT_BAL_AMT'))).alias('CT_CHK_MED_DEPOSIT_BAL_AMT'),
udf_median(F.collect_list(F.col('SAVINGS_BAL_AMT'))).alias('CT_CHK_MED_SAVINGS_BAL_AMT'),
udf_median(F.collect_list(F.col('CHECKING_BAL_AMT'))).alias('CT_CHK_MED_CHECKING_BAL_AMT'),
udf_median(F.collect_list(F.col('TOTAL_CROSS_SELL'))).alias('CT_CHK_MED_TOTAL_CROSS_SELL'),
udf_median(F.collect_list(F.col('INVESTMENT_BAL_AMT'))).alias('CT_CHK_MED_INVESTMENT_BAL_AMT'))
                                   

                                               
agged_BG_CHK =mdm.filter(mdm.CHECKING_IND==1).groupby('DATE_KEY','block_group').agg(F.mean('CONS_BAL_AMT').alias('BG_CHK_MEAN_CONS_BAL_AMT'),
F.mean('DEPOSIT_BAL_AMT').alias('BG_CHK_MEAN_DEPOSIT_BAL_AMT'),
F.mean('SAVINGS_BAL_AMT').alias('BG_CHK_MEAN_SAVINGS_BAL_AMT'),
F.mean('CHECKING_BAL_AMT').alias('BG_CHK_MEAN_CHECKING_BAL_AMT'),
F.mean('TOTAL_CROSS_SELL').alias('BG_CHK_MEAN_TOTAL_CROSS_SELL'),
F.mean('INVESTMENT_BAL_AMT').alias('BG_CHK_MEAN_INVESTMENT_BAL_AMT'),
udf_median(F.collect_list(F.col('CONS_BAL_AMT'))).alias('BG_CHK_MED_CONS_BAL_AMT'),
udf_median(F.collect_list(F.col('DEPOSIT_BAL_AMT'))).alias('BG_CHK_MED_DEPOSIT_BAL_AMT'),
udf_median(F.collect_list(F.col('SAVINGS_BAL_AMT'))).alias('BG_CHK_MED_SAVINGS_BAL_AMT'),
udf_median(F.collect_list(F.col('CHECKING_BAL_AMT'))).alias('BG_CHK_MED_CHECKING_BAL_AMT'),
udf_median(F.collect_list(F.col('TOTAL_CROSS_SELL'))).alias('BG_CHK_MED_TOTAL_CROSS_SELL'),
udf_median(F.collect_list(F.col('INVESTMENT_BAL_AMT'))).alias('BG_CHK_MED_INVESTMENT_BAL_AMT'))

agged_CT_CHK_LT12 =mdm.filter( (mdm.CHECKING_IND==1) & (mdm.LESS_12_MONTHS_TENURE==1) ).groupby('DATE_KEY','census_tract').agg(F.mean('CONS_BAL_AMT').alias('CT_CHK_LT12_MEAN_CONS_BAL_AMT'),
F.mean('DEPOSIT_BAL_AMT').alias('CT_CHK_LT12_MEAN_DEPOSIT_BAL_AMT'),
F.mean('SAVINGS_BAL_AMT').alias('CT_CHK_LT12_MEAN_SAVINGS_BAL_AMT'),
F.mean('CHECKING_BAL_AMT').alias('CT_CHK_LT12_MEAN_CHECKING_BAL_AMT'),
F.mean('TOTAL_CROSS_SELL').alias('CT_CHK_LT12_MEAN_TOTAL_CROSS_SELL'),
F.mean('INVESTMENT_BAL_AMT').alias('CT_CHK_LT12_MEAN_INVESTMENT_BAL_AMT'),
udf_median(F.collect_list(F.col('CONS_BAL_AMT'))).alias('CT_CHK_LT12_MED_CONS_BAL_AMT'),
udf_median(F.collect_list(F.col('DEPOSIT_BAL_AMT'))).alias('CT_CHK_LT12_MED_DEPOSIT_BAL_AMT'),
udf_median(F.collect_list(F.col('SAVINGS_BAL_AMT'))).alias('CT_CHK_LT12_MED_SAVINGS_BAL_AMT'),
udf_median(F.collect_list(F.col('CHECKING_BAL_AMT'))).alias('CT_CHK_LT12_MED_CHECKING_BAL_AMT'),
udf_median(F.collect_list(F.col('TOTAL_CROSS_SELL'))).alias('CT_CHK_LT12_MED_TOTAL_CROSS_SELL'),
udf_median(F.collect_list(F.col('INVESTMENT_BAL_AMT'))).alias('CT_CHK_LT12_MED_INVESTMENT_BAL_AMT'))


agged_BG_CHK_LT12 =mdm.filter( (mdm.CHECKING_IND==1) & (mdm.LESS_12_MONTHS_TENURE==1) ).groupby('DATE_KEY','block_group').agg(F.mean('CONS_BAL_AMT').alias('BG_CHK_LT12_MEAN_CONS_BAL_AMT'),
F.mean('DEPOSIT_BAL_AMT').alias('BG_CHK_LT12_MEAN_DEPOSIT_BAL_AMT'),
F.mean('SAVINGS_BAL_AMT').alias('BG_CHK_LT12_MEAN_SAVINGS_BAL_AMT'),
F.mean('CHECKING_BAL_AMT').alias('BG_CHK_LT12_MEAN_CHECKING_BAL_AMT'),
F.mean('TOTAL_CROSS_SELL').alias('BG_CHK_LT12_MEAN_TOTAL_CROSS_SELL'),
F.mean('INVESTMENT_BAL_AMT').alias('BG_CHK_LT12_MEAN_INVESTMENT_BAL_AMT'),
udf_median(F.collect_list(F.col('CONS_BAL_AMT'))).alias('BG_CHK_LT12_MED_CONS_BAL_AMT'),
udf_median(F.collect_list(F.col('DEPOSIT_BAL_AMT'))).alias('BG_CHK_LT12_MED_DEPOSIT_BAL_AMT'),
udf_median(F.collect_list(F.col('SAVINGS_BAL_AMT'))).alias('BG_CHK_LT12_MED_SAVINGS_BAL_AMT'),
udf_median(F.collect_list(F.col('CHECKING_BAL_AMT'))).alias('BG_CHK_LT12_MED_CHECKING_BAL_AMT'),
udf_median(F.collect_list(F.col('TOTAL_CROSS_SELL'))).alias('BG_CHK_LT12_MED_TOTAL_CROSS_SELL'),
udf_median(F.collect_list(F.col('INVESTMENT_BAL_AMT'))).alias('BG_CHK_LT12_MED_INVESTMENT_BAL_AMT'))


spark = SparkSession.builder.getOrCreate()

#join together the CT

#change names for join
agged_CT_CHK = agged_CT_CHK.withColumnRenamed("DATE_KEY", "DATE_KEY2").withColumnRenamed("census_tract", "census_tract2")
agged_CT_CHK_LT12= agged_CT_CHK_LT12.withColumnRenamed("DATE_KEY", "DATE_KEY3").withColumnRenamed("census_tract", "census_tract3")


agged_CT.registerTempTable("d1")
agged_CT_CHK.registerTempTable("d2") 
agged_CT_CHK_LT12.registerTempTable("d3")


sqlDf_CT = spark.sql("select a.*,b.*,c.* from d1 a inner join d2 b on a.DATE_KEY=b.DATE_KEY2 and a.census_tract = b.census_tract2 inner join d3 c on a.DATE_KEY=c.DATE_KEY3 and a.census_tract = c.census_tract3")

sqlDf_CT=sqlDf_CT.drop('DATE_KEY2')
sqlDf_CT=sqlDf_CT.drop('DATE_KEY3')
sqlDf_CT=sqlDf_CT.drop('census_tract2')
sqlDf_CT=sqlDf_CT.drop('census_tract3')


mdm_agg_CT=sqlDf_CT.toPandas()
mdm_agg_CT.head()




#join together the BG


#change names for join
agged_BG_CHK = agged_BG_CHK.withColumnRenamed("DATE_KEY", "DATE_KEY2").withColumnRenamed("block_group", "block_group2")
agged_BG_CHK_LT12= agged_BG_CHK_LT12.withColumnRenamed("DATE_KEY", "DATE_KEY3").withColumnRenamed("block_group", "block_group3")


agged_BG.registerTempTable("d4")
agged_BG_CHK.registerTempTable("d5") 
agged_BG_CHK_LT12.registerTempTable("d6")

sqlDf_BG = spark.sql("select a.*,b.*,c.* from d4 a inner join d5 b on a.DATE_KEY=b.DATE_KEY2 and a.block_group = b.block_group2 inner join d6 c on a.DATE_KEY=c.DATE_KEY3 and a.block_group = c.block_group3")


sqlDf_BG=sqlDf_BG.drop('DATE_KEY2')
sqlDf_BG=sqlDf_BG.drop('DATE_KEY3')
sqlDf_BG=sqlDf_BG.drop('block_group2')
sqlDf_BG=sqlDf_BG.drop('block_group3')


mdm_agg_BG=sqlDf_BG.toPandas()
mdm_agg_BG.head()

,DATE_KEY,block_group,BG_MEAN_CONS_BAL_AMT,BG_MEAN_DEPOSIT_BAL_AMT,BG_MEAN_SAVINGS_BAL_AMT,BG_MEAN_CHECKING_BAL_AMT,BG_MEAN_TOTAL_CROSS_SELL,BG_MEAN_INVESTMENT_BAL_AMT,BG_MED_CONS_BAL_AMT,BG_MED_DEPOSIT_BAL_AMT,BG_MED_SAVINGS_BAL_AMT,BG_MED_CHECKING_BAL_AMT,BG_MED_TOTAL_CROSS_SELL,BG_MED_INVESTMENT_BAL_AMT,mdm_fe_bg,mdm_ma_bg,mdm_mm_bg,mdm_ub_bg,BG_CHK_MEAN_CONS_BAL_AMT,BG_CHK_MEAN_DEPOSIT_BAL_AMT,BG_CHK_MEAN_SAVINGS_BAL_AMT,BG_CHK_MEAN_CHECKING_BAL_AMT,BG_CHK_MEAN_TOTAL_CROSS_SELL,BG_CHK_MEAN_INVESTMENT_BAL_AMT,BG_CHK_MED_CONS_BAL_AMT,BG_CHK_MED_DEPOSIT_BAL_AMT,BG_CHK_MED_SAVINGS_BAL_AMT,BG_CHK_MED_CHECKING_BAL_AMT,BG_CHK_MED_TOTAL_CROSS_SELL,BG_CHK_MED_INVESTMENT_BAL_AMT,BG_CHK_LT12_MEAN_CONS_BAL_AMT,BG_CHK_LT12_MEAN_DEPOSIT_BAL_AMT,BG_CHK_LT12_MEAN_SAVINGS_BAL_AMT,BG_CHK_LT12_MEAN_CHECKING_BAL_AMT,BG_CHK_LT12_MEAN_TOTAL_CROSS_SELL,BG_CHK_LT12_MEAN_INVESTMENT_BAL_AMT,BG_CHK_LT12_MED_CONS_BAL_AMT,BG_CHK_LT12_MED_DEPOSIT_BAL_AMT,BG_CHK_LT12_MED_SAVINGS_BAL_AMT,BG_CHK_LT12_MED_CHECKING_BAL_AMT,BG_CHK_LT12_MED_TOTAL_CROSS_SELL,BG_CHK_LT12_MED_INVESTMENT_BAL_AMT
0,30APR2018,040132168312,43404.727273,61.363636,0.000000,61.363636,1.181818,0.000000,11507.0,0.0,0.0,0.0,1.0,0.0,5,4,1,0,25929.0,675.000000,0.00,675.000000,3.0,0.0,25929.0,675.0,0.0,675.0,3.0,0.0,25929.0,675.0,0.0,675.0,3.0,0.0,25929.0,675.0,0.0,675.0,3.0,0.0
1,30APR2018,060375438022,0.000000,5.666667,12.000000,-6.333333,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,17.000000,36.00,-19.000000,3.0,0.0,0.0,17.0,36.0,-19.0,3.0,0.0,0.0,17.0,36.0,-19.0,3.0,0.0,0.0,17.0,36.0,-19.0,3.0,0.0
2,30APR2018,060590755151,29113.750000,3082.777778,0.000000,3082.777778,0.805556,2475.888889,0.0,0.0,0.0,0.0,1.0,0.0,3,7,7,1,0.0,36993.333333,0.00,36993.333333,2.0,0.0,0.0,7522.0,0.0,7522.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,30APR2018,060650427092,52525.200000,0.400000,0.200000,0.200000,1.400000,0.000000,6158.0,0.0,0.0,0.0,1.0,0.0,0,1,2,1,2.0,2.000000,1.00,1.000000,3.0,0.0,2.0,2.0,1.0,1.0,3.0,0.0,2.0,2.0,1.0,1.0,3.0,0.0,2.0,2.0,1.0,1.0,3.0,0.0
4,30APR2018,120110304021,19826.607143,5735.107143,40.892857,114.857143,1.535714,0.000000,10.5,0.0,0.0,0.0,1.0,0.0,0,3,6,16,86835.5,40105.000000,286.25,804.000000,4.5,0.0,4061.0,1454.5,20.5,362.0,5.0,0.0,172.0,172.0,10.0,162.0,4.0,0.0,172.0,172.0,10.0,162.0,4.0,0.0


In [4]:
del mdm

<h3>Small files can hold in memory and process as much as possible until joining later with scoring batches</h3>    
                
                

In [5]:
mdm_mob2=pd.read_csv('/user-home/1014/checking_models/score/july18/final_MOB2.csv',na_values=['.','',' '],dtype={'census_tract' : object, 'block_group' : object })


#2017 works for both campaigns as it is recieved jan 2017 (with data through june 16)
ground_counts=pd.read_excel('/user-home/1014/checking_models/score/july18/Ground_Counts_18.xlsx',dtype={'Block Group Code' : object})
ground_counts=ground_counts.fillna(0)
ground_counts['census_tract']=ground_counts['Block Group Code'].str[:-1]
ground_counts.rename(columns={'Block Group Code':'block_group'}, inplace=True)

#info through June 16
ixi_seg_dep=pd.read_excel('/user-home/1014/checking_models/score/july18/ixi_seg_dep_2018.xlsx')

#TSP BG address count
tsp_addresses_bg=pd.read_csv('/user-home/1014/checking_models/score/july18/TSP_CT_Counts.csv',na_values=['.','',' '],dtype={'full_census' : object})



###################################################################################################################################################################


#counts of households for each IXI seg by BG
gc=ground_counts.values[:,1:-2]
print(gc.shape)



#expected CHECKING accounts and balance by BG

#percentage of each ixi seg with checking
per_check=ixi_seg_dep.values[:,4]
print(per_check.shape)

mean_check_value=ixi_seg_dep.values[:,5]
print(per_check.shape)

#number in BG with chekcing estimate
gc_check_households=np.dot(gc,per_check)
print(gc_check_households.shape)


#broadcast % with checking to each row of household counts
hh_with_checking=gc*per_check.T


#average checking balance * # with checking
gc_check_value_households=np.dot(hh_with_checking,mean_check_value)





#expected SAVINGS accounts and balance by BG


#percentage of each ixi seg with savings
per_sav=ixi_seg_dep.values[:,12]
print(per_sav.shape)

mean_sav_value=ixi_seg_dep.values[:,13]
print(per_sav.shape)

#number in BG with savings estimate
gc_sav_households=np.dot(gc,per_sav)
print(gc_sav_households.shape)


#broadcast % with checking to each row of household counts
hh_with_savings=gc*per_sav.T


#average checking balance * # with checking
gc_sav_value_households=np.dot(hh_with_savings,mean_sav_value)



ground_counts['gc_check_households']=gc_check_households
ground_counts['gc_check_balances']=gc_check_value_households
ground_counts['gc_Avg_check_balances']=ground_counts['gc_check_balances'] / ground_counts['gc_check_households']



ground_counts['gc_sav_households']=gc_sav_households
ground_counts['gc_sav_balances']=gc_sav_value_households
ground_counts['gc_Avg_sav_balances']=ground_counts['gc_sav_balances'] / ground_counts['gc_sav_households']

ground_counts['gc_check_sav_balances']=ground_counts['gc_check_balances']+ground_counts['gc_sav_balances']



ground_counts[['block_group','gc_check_households','gc_check_balances','gc_Avg_check_balances','gc_sav_households','gc_sav_balances','gc_Avg_sav_balances','gc_check_sav_balances']].head()

###############################################################################################################################################################################################################

#IXI agg count
def groupIXI(group):
    
    
    fe=(group['I32']+
group['J33']+
group['J34']+
group['J35']+
group['J36']+
group['K37']+
group['K38']+
group['K39']+
group['L40']+
group['L41']+
group['L42']+
group['O52']+
group['O53']+
group['R61']).sum()
    
    ma=(
    group['E17']+
group['E18']+
group['E19']+
group['E20']+
group['F21']+
group['F22']+
group['F23']+
group['F24']+
group['G25']+
group['G26']+
group['G27']+
group['G28']+
group['H29']+
group['H30']+
group['H31']+
group['N48']+
group['N49']+
group['N50']+
group['N51']+
group['Q58']+
group['Q59']+
group['Q60']   
    ).sum()
    
    
    mm=(
    group['A03']+
group['A04']+
group['B06']+
group['B08']+
group['C11']+
group['C12']+
group['D14']+
group['D16']+
group['M45']+
group['M46']+
group['M47']+
group['P56']    
    ).sum()    
    
    
    ub=(
    group['A01']+
group['A02']+
group['B05']+
group['B07']+
group['C09']+
group['C10']+
group['D13']+
group['D15']+
group['M43']+
group['M44']+
group['P54']+
group['P55']+
group['P57']    
    ).sum()
    
        
    return(pd.Series([fe,ma,mm,ub]))
    
    
############################################################################################################################################################################

gc_ct_grp=ground_counts.groupby(['census_tract'])
gc_ct_IXI=gc_ct_grp.apply(groupIXI).reset_index()
gc_ct_IXI.columns=['census_tract', 'FE_Count_ct', 'MA_Count_ct', 'MM_Count_ct', 'UB_Count_ct']
gc_bg_grp=ground_counts.groupby(['block_group'])
gc_bg_IXI=gc_bg_grp.apply(groupIXI).reset_index()
gc_bg_IXI.columns=['block_group', 'FE_Count_bg', 'MA_Count_bg', 'MM_Count_bg', 'UB_Count_bg']


###########################################################################################################################################################################


mdm_mob2_grp = mdm_mob2.groupby(['DK_link','census_tract']) 
mdm_mob2_mean_ct=mdm_mob2_grp[['total_balances_MOB']].mean().add_prefix('CTmean_') 
mdm_mob2_mean_ct=mdm_mob2_mean_ct.reset_index()
mdm_mob2_mean_ct.tail()

mdm_mob2_grp = mdm_mob2.groupby(['DK_link','block_group']) 
mdm_mob2_mean_bg=mdm_mob2_grp[['total_balances_MOB']].mean().add_prefix('BGmean_') 
mdm_mob2_mean_bg=mdm_mob2_mean_bg.reset_index()
mdm_mob2_mean_bg.tail()

(216591, 61)
(61,)
(61,)
(216591,)
(61,)
(61,)
(216591,)


,DK_link,block_group,BGmean_total_balances_MOB
35657,30APR2018,720252006002,4228.0
35658,30APR2018,720310501073,106000.0
35659,30APR2018,72035260700,-4.0
35660,30APR2018,72069180800,85.0
35661,30APR2018,780109714001,188.0


In [6]:
tsp=pd.read_csv('/user-home/1014/checking_models/score/july18/TSP.csv',na_values=['.','',' '],usecols=[
    
   
'Person_Seq_No_1_183',
'Advantage_Target_Income_20_876',
'Target_Income_Index_20_882',
'Target_Net_Worth_20_881',
'Advantage_Home_Owner_813',
'Target_Home_Market_Value_894',
'Advantage_Home_Owner_813',
'Current_Loan_to_Value_890',
'Available_home_equity_in_th_899',
    
'Liquid_Resources_1093',
'Gold_Wallets_1100',
'Silver_Wallets_1102',
'Platinum_Wallets_1101',
    
'Wealth_Resources_1089',
'Investment_Resources_1091',
    'Advantage_Length_of_Residen_823',
    'Target_PreMover_20_Model_879'
]
               
               )

tsp=tsp.drop_duplicates()

# Advantage_Target_Income_20_876   
d1={1:1,
2:2,
3:3,
4:4,
5:5,
6:6,
7:7,
8:8,
9:9,
'A':10,
'B':11,
'C':12,
'D':13
}


d2={'Y':1, 'N':0}



tsp['Advantage_Target_Income_20_876'].update( tsp['Advantage_Target_Income_20_876'].map(d1)) 
tsp['Target_Home_Market_Value_894_v']=tsp.Target_Home_Market_Value_894.str[0:4].fillna(0).astype(float)  
tsp['Current_Loan_to_Value_890']=tsp['Current_Loan_to_Value_890'].fillna(0)
tsp['Available_home_equity_in_th_899']=tsp['Available_home_equity_in_th_899'].fillna(0) 
tsp['Advantage_Length_of_Residen_823']=tsp['Advantage_Length_of_Residen_823'].fillna(0) 
tsp['Target_PreMover_20_Model_879']=tsp['Target_PreMover_20_Model_879'].fillna(0) 

tsp['Gold_Wallets_1100'].fillna('N',inplace=True)
tsp['Silver_Wallets_1102'].fillna('N',inplace=True)
tsp['Platinum_Wallets_1101'].fillna('N',inplace=True)


tsp['Gold_Wallets_1100'].update( tsp['Gold_Wallets_1100'].map(d2))  
tsp['Silver_Wallets_1102'].update( tsp['Silver_Wallets_1102'].map(d2)) 
tsp['Platinum_Wallets_1101'].update( tsp['Platinum_Wallets_1101'].map(d2))  

tsp['Liquid_Resources_1093']=tsp['Liquid_Resources_1093'].astype(float)  
tsp['Wealth_Resources_1089']=tsp['Wealth_Resources_1089'].astype(float)  
tsp['Investment_Resources_1091']=tsp['Investment_Resources_1091'].astype(float)  


keep_vars=[
'Person_Seq_No_1_183',
'Advantage_Target_Income_20_876',
'Target_Income_Index_20_882',
'Target_Net_Worth_20_881',
'Advantage_Home_Owner_813',
'Target_Home_Market_Value_894_v',
'Current_Loan_to_Value_890',
'Available_home_equity_in_th_899',
    
'Liquid_Resources_1093',
'Gold_Wallets_1100',
'Silver_Wallets_1102',
'Platinum_Wallets_1101',
    
'Wealth_Resources_1089',
'Investment_Resources_1091',
    'Advantage_Length_of_Residen_823',
    'Target_PreMover_20_Model_879'

    
]

tsp=tsp[keep_vars]
tsp.head()
     
     
     
     
 


/opt/conda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Person_Seq_No_1_183,Advantage_Target_Income_20_876,Target_Income_Index_20_882,Target_Net_Worth_20_881,Advantage_Home_Owner_813,Target_Home_Market_Value_894_v,Current_Loan_to_Value_890,Available_home_equity_in_th_899,Liquid_Resources_1093,Gold_Wallets_1100,Silver_Wallets_1102,Platinum_Wallets_1101,Wealth_Resources_1089,Investment_Resources_1091,Advantage_Length_of_Residen_823,Target_PreMover_20_Model_879
0,00000000400000000023DE,4,55,0,4,153.0,93.0,13.0,1.0,0,0,0,2.0,4.0,5,31
1,0000000040000000005D09,3,42,0,3,102.0,0.0,0.0,1.0,0,0,0,1.0,1.0,5,50
2,0000000040000000007217,3,53,0,2,0.0,0.0,0.0,1.0,0,0,0,1.0,1.0,2,28
3,000000004000000000883E,9,181,7,4,198.0,97.0,6.0,10.0,0,0,0,6.0,11.0,5,60
4,000000004000000000A1A7,8,278,6,4,210.0,0.0,0.0,5.0,0,0,0,6.0,10.0,5,76


<h3>Iterate scoring file</h3>

In [7]:
import lightgbm as lgb 
from sklearn.linear_model import RidgeCV, LassoCV
import patsy
from patsy import dmatrix, build_design_matrices



#MODEL##################################################################################################################################################################################################
mod=joblib.load('/user-home/1014/checking_models/ridge.pkl')
mod2=joblib.load('/user-home/1014/checking_models/lgb.pkl')
mod3=joblib.load('/user-home/1014/checking_models/lasso.pkl')

###########################################Linking date
d_dk ={'10/31/2017': '31OCT2017','12/31/2017': '31DEC2017','01/31/2018': '31JAN2018','04/30/2018': '30APR2018'}

####################FOR MIN AND MAX####################################################################################################################################################################
train=pd.read_csv('/user-home/1014/checking_models/balance_train.csv')
X=train[['Advantage_Target_Income_20_876',
'BG_CHK_MED_DEPOSIT_BAL_AMT',
'CT_CHK_MEAN_CONS_BAL_AMT',
'CT_MEAN_CONS_BAL_AMT',
'CT_MED_DEPOSIT_BAL_AMT',
'gc_Avg_check_balances',
'gc_Avg_sav_balances',
'Investment_Resources_1091',
'Liquid_Resources_1093',
'prop_FE_mdm_ct',
'prop_FE_bg',
'prop_MA_bg',
'prop_MA_ct',
'prop_MA_mdm_bg',
'prop_MA_mdm_ct',
'prop_UB_bg',
'prop_UB_ct',
'prop_UB_mdm_bg',
'prop_UB_mdm_ct',
'Target_Home_Market_Value_894_v',
'Target_Income_Index_20_882',
'Target_Net_Worth_20_881',
'Target_PreMover_20_Model_879',
'Wealth_Resources_1089',
                'Advantage_Length_of_Residen_823',
                'Advantage_Home_Owner_813',
                
                'Silver_Wallets_1102',
                'Gold_Wallets_1100',
                'gc_check_balances'
]]

mins=X.min()
maxs=X.max()

X=X.fillna(0)
X['prop_FEMA_mdm_ct']=X['prop_FE_mdm_ct']+X['prop_MA_mdm_ct']
X['prop_FEMA_bg']=X['prop_FE_bg']+X['prop_MA_bg']


#cubic spline definitions
x1= dmatrix("cr(x, df=3) - 1", {"x": X.Advantage_Target_Income_20_876.values})
x2= dmatrix("cr(x, df=3) - 1", {"x": X.BG_CHK_MED_DEPOSIT_BAL_AMT.values})
x3= dmatrix("cr(x, df=6) - 1", {"x": X.CT_CHK_MEAN_CONS_BAL_AMT.values})
x4= dmatrix("cr(x, df=6) - 1", {"x": X.CT_MEAN_CONS_BAL_AMT.values})
x5= dmatrix("cr(x, df=3) - 1", {"x": X.CT_MED_DEPOSIT_BAL_AMT.values})
x6= dmatrix("cr(x, df=3) - 1", {"x": X.gc_Avg_check_balances.values})
x7= dmatrix("cr(x, df=3) - 1", {"x": X.gc_Avg_sav_balances.values})
x8= dmatrix("cr(x, df=3) - 1", {"x": X.Investment_Resources_1091.values})
x9= dmatrix("cr(x, df=3) - 1", {"x": X.Liquid_Resources_1093.values})
x10= dmatrix("cr(x, df=6) - 1", {"x": X.prop_FE_mdm_ct.values})
x11= dmatrix("cr(x, df=6) - 1", {"x": X.prop_MA_bg.values})
x12= dmatrix("cr(x, df=6) - 1", {"x": X.prop_MA_ct.values})
x13= dmatrix("cr(x, df=6) - 1", {"x": X.prop_MA_mdm_bg.values})
x14= dmatrix("cr(x, df=6) - 1", {"x": X.prop_MA_mdm_ct.values})
x15= dmatrix("cr(x, df=6) - 1", {"x": X.prop_UB_bg.values})
x16= dmatrix("cr(x, df=6) - 1", {"x": X.prop_UB_ct.values})
x17= dmatrix("cr(x, df=6) - 1", {"x": X.prop_UB_mdm_bg.values})
x18= dmatrix("cr(x, df=6) - 1", {"x": X.prop_UB_mdm_ct.values})
x19= dmatrix("cr(x, df=3) - 1", {"x": X.Target_Home_Market_Value_894_v.values})
x20= dmatrix("cr(x, df=6) - 1", {"x": X.Target_Income_Index_20_882.values})
x21= dmatrix("cr(x, df=3) - 1", {"x": X.Target_Net_Worth_20_881.values})
x22= dmatrix("cr(x, df=3) - 1", {"x": X.Target_PreMover_20_Model_879.values})
x23= dmatrix("cr(x, df=3) - 1", {"x": X.Wealth_Resources_1089.values})
x24= dmatrix("cr(x, df=3) - 1", {"x": X.Advantage_Length_of_Residen_823.values})
x25= dmatrix("cr(x, df=3) - 1", {"x": X.Advantage_Home_Owner_813.values})
x26= dmatrix("cr(x, df=6) - 1", {"x": X.prop_FEMA_mdm_ct.values})
x27= dmatrix("cr(x, df=6) - 1", {"x": X.prop_FEMA_bg.values})


del train
del X
########################################################################################################################################################################################################

mailfiles=pd.read_csv('/user-home/1014/checking_models/score/july18/mailfiles.csv',na_values=['.','',' '],dtype={'full_census' : object, 'block_group': object},iterator=True, chunksize=500000)

for batch in mailfiles:
    batch['date_str'].update( batch['date_str'].map(d_dk))
    train=pd.merge(batch,tsp,left_on='Person_Seq_No_1_183', right_on='Person_Seq_No_1_183')
    print(train.shape)

    #################################
    train=pd.merge(train,mdm_agg_CT,how="left",left_on=['full_census','date_str'], right_on=['census_tract','DATE_KEY'])
    train=pd.merge(train,mdm_agg_BG,how="left",left_on=['block_group','date_str'], right_on=['block_group','DATE_KEY'])

    #impute missing with zero
    train['mdm_fe_ct']=train.mdm_fe_ct.fillna(0)
    train['mdm_ma_ct']=train.mdm_ma_ct.fillna(0)
    train['mdm_mm_ct']=train.mdm_mm_ct.fillna(0)
    train['mdm_ub_ct']=train.mdm_ub_ct.fillna(0)

    train['mdm_fe_bg']=train.mdm_fe_bg.fillna(0)
    train['mdm_ma_bg']=train.mdm_ma_bg.fillna(0)
    train['mdm_mm_bg']=train.mdm_mm_bg.fillna(0)
    train['mdm_ub_bg']=train.mdm_ub_bg.fillna(0)


    train['prop_FE_mdm_ct']= train['mdm_fe_ct'] / (train['mdm_fe_ct']+train['mdm_ma_ct']+train['mdm_mm_ct']+train['mdm_ub_ct']+1)
    train['prop_MA_mdm_ct']= train['mdm_ma_ct'] / (train['mdm_fe_ct']+train['mdm_ma_ct']+train['mdm_mm_ct']+train['mdm_ub_ct']+1)
    train['prop_MM_mdm_ct']= train['mdm_mm_ct'] / (train['mdm_fe_ct']+train['mdm_ma_ct']+train['mdm_mm_ct']+train['mdm_ub_ct']+1)
    train['prop_UB_mdm_ct']= train['mdm_ub_ct'] / (train['mdm_fe_ct']+train['mdm_ma_ct']+train['mdm_mm_ct']+train['mdm_ub_ct']+1)

    train['prop_FE_mdm_bg']= train['mdm_fe_bg'] / (train['mdm_fe_bg']+train['mdm_ma_bg']+train['mdm_mm_bg']+train['mdm_ub_bg']+1)
    train['prop_MA_mdm_bg']= train['mdm_ma_bg'] / (train['mdm_fe_bg']+train['mdm_ma_bg']+train['mdm_mm_bg']+train['mdm_ub_bg']+1)
    train['prop_MM_mdm_bg']= train['mdm_mm_bg'] / (train['mdm_fe_bg']+train['mdm_ma_bg']+train['mdm_mm_bg']+train['mdm_ub_bg']+1)
    train['prop_UB_mdm_bg']= train['mdm_ub_bg'] / (train['mdm_fe_bg']+train['mdm_ma_bg']+train['mdm_mm_bg']+train['mdm_ub_bg']+1)

    ############################
    train=pd.merge(train,mdm_mob2_mean_ct,how="left",left_on=['full_census','date_str'], right_on=['census_tract','DK_link'])
    train=pd.merge(train,mdm_mob2_mean_bg,how="left",left_on=['block_group','date_str'], right_on=['block_group','DK_link'])
    print(train.shape)
    
    train['missing_mob_ct']=train['CTmean_total_balances_MOB'].isnull().astype(int)
    train['missing_mob_bg']=train['BGmean_total_balances_MOB'].isnull().astype(int)
    
    train['CTmean_total_balances_MOB']=train.CTmean_total_balances_MOB.fillna(train.CTmean_total_balances_MOB.mean())
    train['BGmean_total_balances_MOB']=train.BGmean_total_balances_MOB.fillna(train.BGmean_total_balances_MOB.mean())    
    
    ##############################
    train=pd.merge(train,gc_ct_IXI, how="left",left_on='full_census', right_on='census_tract')

    train['FE_Count_ct']=train.FE_Count_ct.fillna(0)
    train['MA_Count_ct']=train.MA_Count_ct.fillna(0)
    train['MM_Count_ct']=train.MM_Count_ct.fillna(0)
    train['UB_Count_ct']=train.UB_Count_ct.fillna(0)

    print(train.shape)
    
    
    train['prop_FE_ct']= train['FE_Count_ct'] / (train['FE_Count_ct']+train['MA_Count_ct']+train['MM_Count_ct']+train['UB_Count_ct']+1)
    train['prop_MA_ct']= train['MA_Count_ct'] / (train['FE_Count_ct']+train['MA_Count_ct']+train['MM_Count_ct']+train['UB_Count_ct']+1)
    train['prop_MM_ct']= train['MM_Count_ct'] / (train['FE_Count_ct']+train['MA_Count_ct']+train['MM_Count_ct']+train['UB_Count_ct']+1)
    train['prop_UB_ct']= train['UB_Count_ct'] / (train['FE_Count_ct']+train['MA_Count_ct']+train['MM_Count_ct']+train['UB_Count_ct']+1)
    
    #################################
    train=pd.merge(train,gc_bg_IXI,how="left",left_on='block_group', right_on='block_group')

    train['FE_Count_bg']=train.FE_Count_bg.fillna(0)
    train['MA_Count_bg']=train.MA_Count_bg.fillna(0)
    train['MM_Count_bg']=train.MM_Count_bg.fillna(0)
    train['UB_Count_bg']=train.UB_Count_bg.fillna(0)

    print(train.shape)
    
    train['prop_FE_bg']= train['FE_Count_bg'] / (train['FE_Count_bg']+train['MA_Count_bg']+train['MM_Count_bg']+train['UB_Count_bg']+1)
    train['prop_MA_bg']= train['MA_Count_bg'] / (train['FE_Count_bg']+train['MA_Count_bg']+train['MM_Count_bg']+train['UB_Count_bg']+1)
    train['prop_MM_bg']= train['MM_Count_bg'] / (train['FE_Count_bg']+train['MA_Count_bg']+train['MM_Count_bg']+train['UB_Count_bg']+1)
    train['prop_UB_bg']= train['UB_Count_bg'] / (train['FE_Count_bg']+train['MA_Count_bg']+train['MM_Count_bg']+train['UB_Count_bg']+1)

    
    train=pd.merge(train,ground_counts[['block_group','gc_check_households','gc_check_balances','gc_Avg_check_balances','gc_sav_households','gc_sav_balances','gc_Avg_sav_balances','gc_check_sav_balances']],how="left",left_on='block_group', right_on='block_group')


    train['gc_check_households']=train.gc_check_households.fillna(0)
    train['gc_check_balances']=train.gc_check_balances.fillna(0)
    train['gc_Avg_check_balances']=train.gc_Avg_check_balances.fillna(0)
    train['gc_sav_households']=train.gc_sav_households.fillna(0)
    train['gc_sav_balances']=train.gc_sav_balances.fillna(0)
    train['gc_Avg_sav_balances']=train.gc_Avg_sav_balances.fillna(0)
    train['gc_check_sav_balances']=train.gc_check_sav_balances.fillna(0)

    print(train.shape)

    ids=train.Person_Seq_No_1_183.values
    train=train[['Advantage_Target_Income_20_876',
'BG_CHK_MED_DEPOSIT_BAL_AMT',
'CT_CHK_MEAN_CONS_BAL_AMT',
'CT_MEAN_CONS_BAL_AMT',
'CT_MED_DEPOSIT_BAL_AMT',
'gc_Avg_check_balances',
'gc_Avg_sav_balances',
'Investment_Resources_1091',
'Liquid_Resources_1093',
'prop_FE_mdm_ct',
'prop_FE_bg',
'prop_MA_bg',
'prop_MA_ct',
'prop_MA_mdm_bg',
'prop_MA_mdm_ct',
'prop_UB_bg',
'prop_UB_ct',
'prop_UB_mdm_bg',
'prop_UB_mdm_ct',
'Target_Home_Market_Value_894_v',
'Target_Income_Index_20_882',
'Target_Net_Worth_20_881',
'Target_PreMover_20_Model_879',
'Wealth_Resources_1089',
                'Advantage_Length_of_Residen_823',
                'Advantage_Home_Owner_813',
                
                'Silver_Wallets_1102',
                'Gold_Wallets_1100',
                'gc_check_balances'
]]
    
    
    train['Advantage_Target_Income_20_876']=train['Advantage_Target_Income_20_876'].astype(float)
    train['Silver_Wallets_1102']=train['Silver_Wallets_1102'].astype(float)
    train['Gold_Wallets_1100']=train['Gold_Wallets_1100'].astype(float)
    
    train=train.clip(lower=mins, upper=maxs, axis=1)
    
    train['prop_FEMA_mdm_ct']=train['prop_FE_mdm_ct']+train['prop_MA_mdm_ct']
    train['prop_FEMA_bg']=train['prop_FE_bg']+train['prop_MA_bg']
    
    train=train.fillna(0)
    
    xx1=build_design_matrices([x1.design_info], {"x":train.Advantage_Target_Income_20_876.values })
    xx2=build_design_matrices([x2.design_info], {"x":train.BG_CHK_MED_DEPOSIT_BAL_AMT.values })
    xx3=build_design_matrices([x3.design_info], {"x":train.CT_CHK_MEAN_CONS_BAL_AMT.values })
    xx4=build_design_matrices([x4.design_info], {"x":train.CT_MEAN_CONS_BAL_AMT.values })
    xx5=build_design_matrices([x5.design_info], {"x":train.CT_MED_DEPOSIT_BAL_AMT.values })
    xx6=build_design_matrices([x6.design_info], {"x":train.gc_Avg_check_balances.values })
    xx7=build_design_matrices([x7.design_info], {"x":train.gc_Avg_sav_balances.values })
    xx8=build_design_matrices([x8.design_info], {"x":train.Investment_Resources_1091.values })
    xx9=build_design_matrices([x9.design_info], {"x":train.Liquid_Resources_1093.values })
    xx10=build_design_matrices([x10.design_info], {"x":train.prop_FE_mdm_ct.values })
    xx11=build_design_matrices([x11.design_info], {"x":train.prop_MA_bg.values })
    xx12=build_design_matrices([x12.design_info], {"x":train.prop_MA_ct.values })
    xx13=build_design_matrices([x13.design_info], {"x":train.prop_MA_mdm_bg.values })
    xx14=build_design_matrices([x14.design_info], {"x":train.prop_MA_mdm_ct.values })
    xx15=build_design_matrices([x15.design_info], {"x":train.prop_UB_bg.values })
    xx16=build_design_matrices([x16.design_info], {"x":train.prop_UB_ct.values })
    xx17=build_design_matrices([x17.design_info], {"x":train.prop_UB_mdm_bg.values })
    xx18=build_design_matrices([x18.design_info], {"x":train.prop_UB_mdm_ct.values })
    xx19=build_design_matrices([x19.design_info], {"x":train.Target_Home_Market_Value_894_v.values })
    xx20=build_design_matrices([x20.design_info], {"x":train.Target_Income_Index_20_882.values })
    xx21=build_design_matrices([x21.design_info], {"x":train.Target_Net_Worth_20_881.values })
    xx22=build_design_matrices([x22.design_info], {"x":train.Target_PreMover_20_Model_879.values })
    xx23=build_design_matrices([x23.design_info], {"x":train.Wealth_Resources_1089.values })
    xx24=build_design_matrices([x24.design_info], {"x":train.Advantage_Length_of_Residen_823.values })
    xx25=build_design_matrices([x25.design_info], {"x":train.Advantage_Home_Owner_813.values })
    xx26=build_design_matrices([x26.design_info], {"x":train.prop_FEMA_mdm_ct.values })
    xx27=build_design_matrices([x27.design_info], {"x":train.prop_FEMA_bg.values })



    Z2=np.column_stack(
        [
    np.array(xx1[0]),
    np.array(xx2[0]),
    np.array(xx3[0]),
    np.array(xx4[0]),
    np.array(xx5[0]),
    np.array(xx6[0]),
    np.array(xx7[0]),
    np.array(xx8[0]),
    np.array(xx9[0]),
    np.array(xx10[0]),
    np.array(xx11[0]),
    np.array(xx12[0]),
    np.array(xx13[0]),
    np.array(xx14[0]),
    np.array(xx15[0]),
    np.array(xx16[0]),
    np.array(xx17[0]),
    np.array(xx18[0]),
    np.array(xx19[0]),
    np.array(xx20[0]),
    np.array(xx21[0]),
    np.array(xx22[0]),
    np.array(xx23[0]),
    np.array(xx24[0]),
    np.array(xx25[0]),
    np.array(xx26[0]),
    np.array(xx27[0])
        ]
    )  

    
    pred_test=mod.predict(Z2)
    pred_test2=mod2.predict(train)
    pred_test3=mod3.predict(Z2)

    pred=(pred_test+pred_test2+pred_test3)/3
    
    
    
    
    
    
    
    
    
    
    
    
    d = {'Person_Seq_No_1_183' : ids,   'pred' : pred}

    toExport=pd.DataFrame(d)
    

    if os.path.isfile('/user-home/1014/checking_models/score/july18/scored.csv') :
        toExport.to_csv('/user-home/1014/checking_models/score/july18/scored.csv',header=False,index=False, mode='a')
    else:
        toExport.to_csv('/user-home/1014/checking_models/score/july18/scored.csv',header=True,index=False,mode='a')
    
    print("scored..."+str(toExport.shape[0])+'  rows...')

print("Done.")
    
    













/opt/conda/lib/python2.7/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator RidgeCV from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python2.7/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LassoCV from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


(500000, 22)
(500000, 118)
(500000, 125)
(500000, 133)
(500000, 144)
scored...500000  rows...
(500000, 22)
(500000, 118)
(500000, 125)
(500000, 133)
(500000, 144)
scored...500000  rows...
(500000, 22)
(500000, 118)
(500000, 125)
(500000, 133)
(500000, 144)
scored...500000  rows...
(500000, 22)
(500000, 118)
(500000, 125)
(500000, 133)
(500000, 144)
scored...500000  rows...
(500000, 22)
(500000, 118)
(500000, 125)
(500000, 133)
(500000, 144)
scored...500000  rows...
(500000, 22)
(500000, 118)
(500000, 125)
(500000, 133)
(500000, 144)
scored...500000  rows...
(500000, 22)
(500000, 118)
(500000, 125)
(500000, 133)
(500000, 144)
scored...500000  rows...
(500000, 22)
(500000, 118)
(500000, 125)
(500000, 133)
(500000, 144)
scored...500000  rows...
(500000, 22)
(500000, 118)
(500000, 125)
(500000, 133)
(500000, 144)
scored...500000  rows...
(500000, 22)
(500000, 118)
(500000, 125)
(500000, 133)
(500000, 144)
scored...500000  rows...
(500000, 22)
(500000, 118)
(500000, 125)
(500000, 133)
(5000

In [8]:
scores=pd.read_csv('/user-home/1014/checking_models/score/july18/scored.csv')
scores.describe()

,pred
count,4.117774e+07
mean,5.231908e+00
std,8.098397e-01
min,2.430535e+00
25%,4.611393e+00
50%,5.269000e+00
75%,5.869853e+00
max,7.618189e+00


In [11]:
#last time scores.describe()

,pred
count,4.046704e+07
mean,5.165612e+00
std,7.852756e-01
min,2.450294e+00
25%,4.569591e+00
50%,5.224734e+00
75%,5.777143e+00
max,7.474721e+00


In [9]:
#logged
#decile 1, 2-7, 8-10

def decile_classify(x):
    if x >= 6.101732:
        decile_range='High_High'
    elif x < 6.101732 and x >= 5.622079:
        decile_range="High"
    elif x < 5.622079 and x >= 5.195267:
        decile_range="High-medium"
    elif x < 5.195267 and x >= 4.215497:
        decile_range="Medium"
    elif x < 4.215497:
        decile_range ="Low"
    else:
        decile_range ="?"
    return(decile_range)



scores['decile_range']=scores.pred.apply(decile_classify)

In [11]:
scores['decile_range'].value_counts()

Medium         14223428
High            8419220
High-medium     7027793
High_High       6318153
Low             5189142
Name: decile_range, dtype: int64

In [10]:
#scores['decile_range'].value_counts()

Medium         14037564
High            8381125
High-medium     7289381
Low             5394533
High_High       5208570
Name: decile_range, dtype: int64

In [13]:
#last time scores['decile_range'].value_counts()

Medium         14172283
High            8557626
High-medium     7578797
Low             5583328
High_High       4575004
Name: decile_range, dtype: int64

In [14]:
scores['decile_range'].describe()

count     40467038
unique           5
top         Medium
freq      14172283
Name: decile_range, dtype: object

In [7]:
15786924+10088144+8640867+5401964+549139

40467038

In [11]:
nw=pd.merge(left=tsp[['Target_Net_Worth_20_881','Person_Seq_No_1_183','Liquid_Resources_1093','Gold_Wallets_1100','Silver_Wallets_1102','Platinum_Wallets_1101']],right=scores,left_on=['Person_Seq_No_1_183'], right_on=['Person_Seq_No_1_183']) 

#del tsp 

nw['NetWorthGT3']=np.where(nw.Target_Net_Worth_20_881.values >=3,1,0)
nw['NetWorthGT4']=np.where(nw.Target_Net_Worth_20_881.values >=4,1,0)
nw['NetWorthGT5']=np.where(nw.Target_Net_Worth_20_881.values >=5,1,0)

nw['LRGT4']=np.where(nw.Liquid_Resources_1093 >=4,1,0)
nw['LRGT5']=np.where(nw.Liquid_Resources_1093.values >=5,1,0)
nw['LRGT6']=np.where(nw.Liquid_Resources_1093 >=6,1,0)

In [12]:
nw.groupby('decile_range')[['NetWorthGT3', 'NetWorthGT4', 'NetWorthGT5','Gold_Wallets_1100','Silver_Wallets_1102','Platinum_Wallets_1101','LRGT4','LRGT5','LRGT6']].mean()

,NetWorthGT3,NetWorthGT4,NetWorthGT5,LRGT4,LRGT5,LRGT6
decile_range,,,,,,
High,0.976517,0.954321,0.867551,0.921408,0.856986,0.786919
High-medium,0.800535,0.688188,0.477667,0.586684,0.450602,0.353015
High_High,0.999345,0.998070,0.984706,0.993491,0.985844,0.973864
Low,0.004649,0.001911,0.000399,0.001750,0.000545,0.000225
Medium,0.201602,0.127139,0.057970,0.109052,0.061080,0.037783


In [ ]:
#jan 18 response model
def resp_decile_classify(x):
    if x >= 0.0217119558105546:
        resp_decile_range=1
    elif x >= 0.0197263513809035 and x <0.0217119558105546 :
        resp_decile_range=2
    elif  x >= 0.0184484491378057 and x <0.0197263513809035 :
        resp_decile_range=3
    elif x >= 0.0174470925486009 and x <0.0184484491378057 :
        resp_decile_range=4
    elif x >= 0.016532322879462 and x <0.0174470925486009 :
        resp_decile_range=5
    elif x >= 0.0156459685112022 and x <0.016532322879462 :
        resp_decile_range=6
    elif x >= 0.0147148677886371 and x <0.0156459685112022 :
        resp_decile_range=7
    elif x >= 0.0136301639856794 and x <0.0147148677886371 :
        resp_decile_range=8
    elif x >= 0.0120405018210083 and x <0.0136301639856794 :
        resp_decile_range=9
    else x < 0.0120405018210083:
        resp_decile_range=10
    else:
        resp_decile_range = 11
 

    return(resp_decile_range)



scores['decile_range']=scores.pred.apply(decile_classify)

In [2]:
import xgboost as xgb
from sklearn.externals import joblib

mod=joblib.load('/user-home/1014/checking_models/ridge.pkl')
mod2=joblib.load('/user-home/1014/checking_models/lgb.pkl')
mod3=joblib.load('/user-home/1014/checking_models/lasso.pkl')

array([-0.48632194,  0.02662359,  0.45969834, -0.07014232, -0.17774505,
        0.24788737,  0.22946083,  0.13055767,  0.23213124, -0.19617608,
       -0.34774258, -0.04823107, -0.18689158, -0.02716541, -0.11546983,
        0.14721719,  0.21431795, -0.03200833, -0.28501699, -0.28339502,
        0.56841201, -0.42157964,  0.60133496, -0.17975532,  0.56454627,
       -0.43571432, -0.12883196, -0.0503911 ,  0.35926905, -0.30887794,
       -0.57272273,  0.13799615,  0.43472658, -0.27223147, -0.12783283,
       -0.09247387, -0.00607996,  0.15521951,  0.34339862,  0.1847536 ,
       -0.34597376,  0.1433823 , -0.04947396,  0.15329222, -0.08598041,
       -0.23521099,  0.2797944 , -0.0926738 ,  0.0532553 ,  0.22409567,
       -0.22926057, -0.31257799, -0.1309296 , -0.14203417,  0.04633024,
       -0.26094026,  0.80015178, -0.27765173, -0.00416234,  0.06939003,
       -0.15949115,  0.16426634,  0.20764886,  0.46016578,  0.08580569,
        0.24243466, -0.02631158,  0.05873472, -0.82082927,  0.01